In [1]:
import json
import lzma
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import seaborn as sns
from collections import Counter
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from pandas.io.json import json_normalize

In [16]:
from bs4 import BeautifulSoup
import lxml

In [3]:
base_path = "./data/xml"
state='north_carolina.xz'
f = lzma.open(os.path.join(base_path,state),"rb")
state_data = f.readlines()
f.close()

In [4]:
data_json = [json.loads(line) for line in state_data]
print(f'Flattening data for {state}')
data = json_normalize(data_json)

Flattening data for north_carolina.xz


C:\Users\gufra\.conda\envs\cs109b\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
data['decision_date_p'] = pd.to_datetime(data.decision_date,errors='coerce')
data['decision_year'] = data.decision_date_p.dt.year

In [6]:
data_2015 = data[data.decision_year==2015]

In [7]:
data_2015.head()

,id,url,name,name_abbreviation,decision_date,docket_number,first_page,last_page,citations,cites_to,...,jurisdiction.name_long,jurisdiction.name,jurisdiction.whitelisted,jurisdiction.id,jurisdiction.url,jurisdiction.slug,casebody.status,casebody.data,decision_date_p,decision_year
92741,4329281,https://api.capapi.org/v1/cases/4329281/,State v. Robert McPhail,State v. McPhail,2015-01-22,341P14,811,811,"[{'type': 'official', 'cite': '367 N.C. 811'}]",[],...,North Carolina,N.C.,True,5,https://api.capapi.org/v1/jurisdictions/nc/,nc,ok,<?xml version='1.0' encoding='utf-8'?>\n<caseb...,2015-01-22,2015.0
92742,4318618,https://api.capapi.org/v1/cases/4318618/,STATE OF NORTH CAROLINA v. TORREY DALE GRADY,State v. Grady,2015-06-10,No. 179A14-2,821,821,"[{'type': 'official', 'cite': '367 N.C. 821'}]","[{'cite': '191 L. Ed. 2d 459'}, {'cite': '135 ...",...,North Carolina,N.C.,True,5,https://api.capapi.org/v1/jurisdictions/nc/,nc,ok,<?xml version='1.0' encoding='utf-8'?>\n<caseb...,2015-06-10,2015.0
92758,4332768,https://api.capapi.org/v1/cases/4332768/,"Clorey Eugene France v. Glover & Petersen, P.A...","France v. Glover & Petersen, P.A.",2015-01-22,452P14,813,813,"[{'type': 'official', 'cite': '367 N.C. 813'}]",[],...,North Carolina,N.C.,True,5,https://api.capapi.org/v1/jurisdictions/nc/,nc,ok,<?xml version='1.0' encoding='utf-8'?>\n<caseb...,2015-01-22,2015.0
92770,4325043,https://api.capapi.org/v1/cases/4325043/,State v. Joshua Andrew Stepp,State v. Stepp,2015-01-22,038A14,809,809,"[{'type': 'official', 'cite': '367 N.C. 809'}]",[],...,North Carolina,N.C.,True,5,https://api.capapi.org/v1/jurisdictions/nc/,nc,ok,<?xml version='1.0' encoding='utf-8'?>\n<caseb...,2015-01-22,2015.0
92775,4323615,https://api.capapi.org/v1/cases/4323615/,State v. Raymond Dakim Harris Joiner,State v. Joiner,2015-01-22,514P13-3,814,814,"[{'type': 'official', 'cite': '367 N.C. 814'}]",[],...,North Carolina,N.C.,True,5,https://api.capapi.org/v1/jurisdictions/nc/,nc,ok,<?xml version='1.0' encoding='utf-8'?>\n<caseb...,2015-01-22,2015.0


In [8]:
markup = data_2015[data_2015.id==12208330]['casebody.data'].iloc[0]

In [9]:
soup = BeautifulSoup(markup, "xml")

In [10]:
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<casebody firstpage="563" lastpage="567" xmlns="http://nrs.harvard.edu/urn-3:HLS.Libr.US_Case_Law.Schema.Case_Body:v1">
 <parties id="b713-7">
  STEPHANIE L. NEEDHAM, Individually and as Guardian ad Litem for John Doe, Jane Doe, and June Doe, minor children v. ROY ALAN PRICE
 </parties>
 <docketnumber id="AWu_">
  No. 81PA15
 </docketnumber>
 <decisiondate id="b713-10">
  Filed 18 December 2015
 </decisiondate>
 <headnotes id="b713-11">
  Parent and Child — parent-child doctrine — unemancipated minors— not a bar to gross negligence, intentional infliction of emotional distress, or punitive damages — bars ordinary negligence
 </headnotes>
 <headnotes id="AeA">
  The Court of Appeals erred in a negligence, premises liability, negligent infliction of emotional distress, intentional infliction of emotional distress, gross negligence, and punitive damages case by reversing in part the trial court’s order granting summary judgment in favor of defendant.

In [15]:
[headnote.text for headnote in soup.find_all('headnotes')]

['Parent and Child — parent-child doctrine — unemancipated minors— not a bar to gross negligence, intentional infliction of emotional distress, or punitive damages — bars ordinary negligence',
 'The Court of Appeals erred in a negligence, premises liability, negligent infliction of emotional distress, intentional infliction of emotional distress, gross negligence, and punitive damages case by reversing in part the trial court’s order granting summary judgment in favor of defendant. The parent-child immunity doctrine neither bars unemancipated minors’ claims based on gross negligence and intentional infliction of emotional distress nor defeats their claims for punitive damages. The parent-child immunity doctrine ‘bars actions between unemancipated children and their parents based on ordinary negligence.']